## 5.1 缺失值比例>80%:根据80%法则 (Bijlsma et al. 2006)：当某一物质的非缺失部分低于总样本量的80%时，建议删除该物质.删除缺失值占比超过80%的特征列

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
hu_data = pd.read_csv(r"C:\Users\12045\Desktop\户数据(预处理中).csv")
ground_data = pd.read_csv(r"C:\Users\12045\Desktop\地块数据(预处理中).csv")
village_data = pd.read_csv(r"C:\Users\12045\Desktop\村数据(预处理中).csv")

# 删除村数据中缺失值占比超过80%的特征列
missing_percentage = (village_data.isnull().sum() / len(village_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 80].index
village_data = village_data.drop(columns=columns_with_high_missing_percentage)

# 删除地块数据中缺失值占比超过80%的特征列
missing_percentage = (ground_data.isnull().sum() / len(ground_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 80].index
ground_data = ground_data.drop(columns=columns_with_high_missing_percentage)

# 删除户数据中缺失值占比超过80%的特征列
missing_percentage = (hu_data.isnull().sum() / len(hu_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 80].index
hu_data = hu_data.drop(columns=columns_with_high_missing_percentage)

# 5.2处理户数据

## 5.2.1 缺失值比例<80%:采用填充方法,比较均值填充，KNN填充，多重插补，填充后用决策树回归器计算四种模型MAE，选择最优填充模型。

In [2]:
from sklearn.experimental import enable_iterative_imputer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import accuracy_score
import numpy as np
# 用租金列不含缺失值的数据训练
# 删除 ground_data 中 "zujin" 列缺失的样本
hu_data_without_missing_zujin = hu_data.dropna(subset=["zujin"])
# 用均值填充
mean_imputer = SimpleImputer(strategy='mean')
hu_data_mean = mean_imputer.fit_transform(hu_data_without_missing_zujin)

# 用K近邻插补填充
knn_imputer = KNNImputer()
hu_data_knn = knn_imputer.fit_transform(hu_data_without_missing_zujin)

# 用多变量特征插补填充（多重插补）
iterative_imputer = IterativeImputer(random_state=42)
hu_data_ite = iterative_imputer.fit_transform(hu_data_without_missing_zujin)

# 检验是否全部填充完整
print("Mean Imputation - Missing Values Count:", np.isnan(hu_data_mean).sum())
print("KNN Imputation - Missing Values Count:", np.isnan(hu_data_knn).sum())
print("Iterative Imputation - Missing Values Count:", np.isnan(hu_data_ite).sum())


Mean Imputation - Missing Values Count: 0
KNN Imputation - Missing Values Count: 0
Iterative Imputation - Missing Values Count: 0


In [3]:
# 用学习后的决策树回归器去预测测试集的标签值，和原始数据集的标签“zujin”对比，计算四种方式的MAE
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
# 定义标签列
# 确定 'zujin' 列在第几列
zujin_column_index = None
for i, column in enumerate(hu_data.columns):
    if column == 'zujin':
        zujin_column_index = i
        break

# 提取 'zujin' 列作为标签
labels_mean = hu_data_mean[:, zujin_column_index] 
labels_knn = hu_data_knn[:, zujin_column_index] 
labels_ite = hu_data_ite[:, zujin_column_index] 
# 定义决策树回归器
tree_regressor = DecisionTreeRegressor()

# 使用五折交叉验证计算 Mean Imputation 的 MAE
mae_mean_cv = cross_val_score(tree_regressor, hu_data_mean, labels_mean, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv = -mae_mean_cv.mean()  # 取负号得到正的 MAE

# 使用五折交叉验证计算 KNN Imputation 的 MAE
mae_knn_cv = cross_val_score(tree_regressor, hu_data_knn, labels_knn, cv=10, scoring='neg_mean_absolute_error')
mae_knn_cv = -mae_knn_cv.mean()  # 取负号得到正的 MAE

# 使用五折交叉验证计算 Iterative Imputation 的 MAE
mae_ite_cv = cross_val_score(tree_regressor, hu_data_ite, labels_ite, cv=10, scoring='neg_mean_absolute_error')
mae_ite_cv = -mae_ite_cv.mean()  # 取负号得到正的 MAE

# 输出交叉验证得到的平均绝对误差
print("Mean Imputation MAE :", mae_mean_cv)
print("KNN Imputation MAE :", mae_knn_cv)
print("Iterative Imputation MAE :", mae_ite_cv)


Mean Imputation MAE : 54.298523391812864
KNN Imputation MAE : 67.20261695906433
Iterative Imputation MAE : 62.372573099415206


#### 结果显示多数情况下户数据采用均值填充更优

## 5.2.2对填充方法优化：额外处理缺失值比例>50%并且<80%的列。

### 对比删除缺失值比例>50%并且<80%的列后用均值填充，删除缺失值比例>80%的列后用均值填充，删除缺失值比例>50%并且<80%的列后线性回归和均值结合填充

In [4]:
# 删除缺失值比例>50%的列后用均值填充，删除缺失值比例>50%的列后用均值填充对比
from sklearn.experimental import enable_iterative_imputer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
hu_data = pd.read_csv(r"C:\Users\12045\Desktop\户数据(预处理中).csv")
# 删除户数据中缺失值占比超过80%的特征列
missing_percentage = (hu_data.isnull().sum() / len(hu_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 80].index
hu_data80 = hu_data.drop(columns=columns_with_high_missing_percentage)
# 删除户数据中缺失值占比超过50%的特征列
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 50].index
hu_data50 = hu_data.drop(columns=columns_with_high_missing_percentage)

# 用租金列不含缺失值的数据训练
# 删除 "zujin" 列缺失的样本
hu_data_without_missing_zujin80 = hu_data80.dropna(subset=["zujin"])
hu_data_without_missing_zujin50 = hu_data50.dropna(subset=["zujin"])
# 用均值填充
mean_imputer = SimpleImputer(strategy='mean')
hu_data_mean80 = mean_imputer.fit_transform(hu_data_without_missing_zujin80)
hu_data_mean50 = mean_imputer.fit_transform(hu_data_without_missing_zujin50)
# 检验是否全部填充完整
print("Mean Imputation80 - Missing Values Count:", np.isnan(hu_data_mean80).sum())
print("Mean Imputation50 - Missing Values Count:", np.isnan(hu_data_mean50).sum())


Mean Imputation80 - Missing Values Count: 0
Mean Imputation50 - Missing Values Count: 0


In [5]:
zujin_column_index80 = None
for i, column in enumerate(hu_data80.columns):
    if column == 'zujin':
        zujin_column_index80 = i
        break
zujin_column_index50 = None
for i, column in enumerate(hu_data50.columns):
    if column == 'zujin':
        zujin_column_index50 = i
        break
# 提取 'zujin' 列作为标签
labels_mean80 = hu_data_mean80[:, zujin_column_index80] 
labels_mean50 = hu_data_mean50[:, zujin_column_index50]
# 定义决策树回归器
tree_regressor = DecisionTreeRegressor()

# 使用五折交叉验证计算 Mean Imputation80 的 MAE
mae_mean_cv80 = cross_val_score(tree_regressor, hu_data_mean80, labels_mean80, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv80 = -mae_mean_cv80.mean()  # 取负号得到正的 MAE

# 使用五折交叉验证计算 Mean Imputation50 的 MAE
mae_mean_cv50 = cross_val_score(tree_regressor, hu_data_mean50, labels_mean50, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv50 = -mae_mean_cv50.mean()  # 取负号得到正的 MAE
print("Mean Imputation MAE80 :", mae_mean_cv80)
print("Mean Imputation MAE50 :", mae_mean_cv50)

Mean Imputation MAE80 : 58.59596491228069
Mean Imputation MAE50 : 69.13119883040936


### 结果显示多数情况下删除缺失值比例大于50的特征列效果更好

### 接下来对比删除缺失值比例>50%的列后用均值填充，线性回归和均值结合填充.

#### 实现线性回归和均值结合填充:有单一其他特征与目标特征相关性> 0.5则采用线性回归填充，没有相关性>0.5的特征则用均值填充。

In [7]:
# 计算相关性
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import numpy as np
hu_data = pd.read_csv(r"C:\Users\12045\Desktop\户数据(预处理中).csv")
# 删除户数据中缺失值占比超过50%的特征列
missing_percentage = (hu_data.isnull().sum() / len(hu_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 50].index
hu_data50 = hu_data.drop(columns=columns_with_high_missing_percentage)
hu_data50 = hu_data50.dropna(subset=["zujin"])
# 复制数据集
Corr_data = hu_data50.copy()
# 各字段间相关程度
correlation = Corr_data.corr()
# 创建一个空字典来保存每个字段与其相似性最高的另一字段名称及相似性
similarities = {}
# 遍历数据框的列
for column in Corr_data.columns:
    # 计算该列与其他列的相关性
    correlations = Corr_data.corr()[column].drop(column)
    # 找到与当前列相关性最高的字段名和相似性值
    most_similar_column = correlations.idxmax()
    highest_correlation = correlations.max()
    # 保存到字典中
    similarities[column] = (most_similar_column, highest_correlation)
# 输出结果
for column, (most_similar_column, highest_correlation) in similarities.items():
    print(f"字段 '{column}' 与字段 '{most_similar_column}' 的相似性最高，相关性为: {highest_correlation}")

字段 'hid' 与字段 'year' 的相似性最高，相关性为: 0.10879444549061598
字段 'year' 与字段 's_chengbaogd' 的相似性最高，相关性为: 0.2226445111781447
字段 'chengbaogd' 与字段 'jycbgdarea' 的相似性最高，相关性为: 0.5577771450189338
字段 'chengbaogdks' 与字段 'zcgdksnum' 的相似性最高，相关性为: 0.8109860784317756
字段 'jygdarea' 与字段 'jynhgdarea' 的相似性最高，相关性为: 0.9248791762704616
字段 'jycbgdarea' 与字段 'chengbaogd' 的相似性最高，相关性为: 0.5577771450189338
字段 'jyczgdarea' 与字段 'c_chengbaogd' 的相似性最高，相关性为: 0.1301735693095028
字段 'jynhgdarea' 与字段 'jygdarea' 的相似性最高，相关性为: 0.9248791762704616
字段 'jygdnum' 与字段 'jygdnum1' 的相似性最高，相关性为: 0.5297853720663166
字段 'jygdnum1' 与字段 'jygdnum' 的相似性最高，相关性为: 0.5297853720663166
字段 'jygdnum5' 与字段 'jynhgdarea' 的相似性最高，相关性为: 0.5517633067772666
字段 'zcgdarea' 与字段 'zcareahetong' 的相似性最高，相关性为: 0.7171671711749427
字段 'zcgdksnum' 与字段 'chengbaogdks' 的相似性最高，相关性为: 0.8109860784317756
字段 'zcqiyearea' 与字段 'zcgdksnum' 的相似性最高，相关性为: 0.15873951328610372
字段 'zchezuoshearea' 与字段 'zcgdarea' 的相似性最高，相关性为: 0.15643115947966557
字段 'zccunjitiarea' 与字段 'zcgdarea' 的相似性最高，相关性为: 0.4

In [8]:
# 定义填充函数
def fill_missing_values(df, column, similar_column, data):
    if df[column].isnull().any():  # 检查目标列是否有缺失值
        correlation = data

        if correlation < 0.5 or df[similar_column].isnull().any():
            # 使用均值填充
            print(f"列'{column}'均值填充")
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        else:
            # 使用线性回归预测填充
            print(f"列'{column}'线性回归预测填充")
            known_data = df[[column, similar_column]].dropna()
            unknown_data = df[df[column].isnull()]
            
            # 定义特征和目标列
            features = [similar_column]
            target_columns = [column]

            # 划分训练集和测试集
            X_train  = known_data[features] 
            y_train=known_data[target_columns]
            # 训练线性回归模型
            model = LinearRegression()
            model.fit(X_train, y_train)

            # 预测缺失值
            predicted_values = model.predict(unknown_data[features])

            # 填充缺失值
            unknown_data[target_columns] = predicted_values
            filled_data = known_data._append(unknown_data)
            
            df.loc[filled_data.index, [column, similar_column]] = filled_data[[column, similar_column]]

# 遍历每个字段
for key, (first_element, second_element) in similarities.items():
    fill_missing_values(Corr_data, key, first_element, second_element)
# 检查是否还有缺失值
missing_values = Corr_data.isnull().sum().sum()
if missing_values == 0:
    print("数据中不存在任何缺失值。")
else:
    print(f"数据中仍有缺失值。")

列'hid'均值填充
列'chengbaogdks'均值填充
列'jygdarea'均值填充
列'jycbgdarea'线性回归预测填充
列'jyczgdarea'均值填充
列'jynhgdarea'线性回归预测填充
列'jygdnum'均值填充
列'jygdnum1'线性回归预测填充
列'jygdnum5'线性回归预测填充
列'zcgdarea'均值填充
列'zcgdksnum'线性回归预测填充
列'zcqiyearea'均值填充
列'zchezuoshearea'均值填充
列'zccunjitiarea'均值填充
列'zcnonghuarea'线性回归预测填充
列'zcwainonghuarea'均值填充
列'zcareahetong'线性回归预测填充
列'zcareaqixian'线性回归预测填充
列'jitizcarea'均值填充
列'jitizcareacgb'线性回归预测填充
列'jitizcareazj'均值填充
列'zcareashouzu'均值填充
列'cbdfangwei'均值填充
列'cbdarea'均值填充
列'cbdcwjuli'均值填充
列'cbddljuli'均值填充
列'cbdgaosulu'均值填充
列'cbdguangai'均值填充
列'cbdfeili'均值填充
列'cbdwrxiufu'均值填充
列'shifouzc'均值填充
数据中不存在任何缺失值。


In [9]:
from sklearn.tree import DecisionTreeRegressor
# 提取 'zujin' 列作为标签
labels = hu_data50["zujin"]
# 定义决策树回归器
tree_regressor = DecisionTreeRegressor()

mae_mean_cv_line = cross_val_score(tree_regressor, Corr_data, labels, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv_line = -mae_mean_cv_line.mean()  # 取负号得到正的 MAE

mae_mean_cv50 = cross_val_score(tree_regressor, hu_data_mean50, labels, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv50 = -mae_mean_cv50.mean()  # 取负号得到正的 MAE
print("均值和线性回归结合填充 :", mae_mean_cv_line)
print("均值填充 :", mae_mean_cv50)

均值和线性回归结合填充 : 67.6263596491228
均值填充 : 53.07527777777777


## 5.2.3最终填充：结果显示采用线性回归和均值填充结合的方法效果更好，如果线性回归预测填充出现了负数，那对这一列改成采用均值填充，分类数据采用众数填充

In [10]:
# 用线性回归结合均值填充
# 删除户数据中缺失值占比超过50%的特征列
missing_percentage = (hu_data.isnull().sum() / len(hu_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 50].index
hu_data50 = hu_data.drop(columns=columns_with_high_missing_percentage)
# 对分类数据缺失值采用众数填充
columns_to_fill = ["cbdfangwei",
                    "cbdpodu", "zrdpodu", "cbdgaosulu", 
                    "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", 
                    "zrdguangai", "cbdfeili", "zrdfeili", "cbdyongtu201912",
                    "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", 
                    "cbdwrxiufu", "zrdwrxiufu", "shifouzc", "cbdjinqin", 
                    "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao", 
                    "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt",
                    "zrdjypt", "cbdjyptreason1", "zrdjyptreason1",
                    "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", 
                    "zrdjiangqixian", "d301a", "d301b", "d311a", "d311b", 
                    "d312a", "d312b", "d313a", "d313b", "d314a", "d314b", 
                    "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", 
                    "d319a", "d319b", "d320a", "d320b", "d321a", "d321b", 
                    "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", 
                    "d325a", "d325b", "d328a", "d328b", "d329a", "d329b", 
                    "d331a", "d331b", "d335a", "d335b", "d336a", "d336b", 
                    "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", 
                    "d340a", "d340b", "d341a", "d341b","jygdnum","jygdnum1","jygdnum5",
                    "zcgdksnum","zcgdksnum","chengbaogdks"]

# 填充缺失值
for column in hu_data50:
    if column in columns_to_fill:
        mode_val = hu_data50[column].mode()[0]  # 计算众数并取第一个值
        hu_data50[column].fillna(mode_val, inplace=True)
# 复制数据集
Corr_data = hu_data50.copy()
# 各字段间相关程度
correlation = Corr_data.corr()
# 创建一个空字典来保存每个字段与其相似性最高的另一字段名称及相似性
similarities = {}
# 遍历数据框的列
for column in Corr_data.columns:
    # 计算该列与其他列的相关性
    correlations = Corr_data.corr()[column].drop(column)
    # 找到与当前列相关性最高的字段名和相似性值
    most_similar_column = correlations.idxmax()
    highest_correlation = correlations.max()
    # 保存到字典中
    similarities[column] = (most_similar_column, highest_correlation)
# 定义填充函数
def fill_missing_values(df, column, similar_column, data):
    if df[column].isnull().any():  # 检查目标列是否有缺失值
        correlation = data

        if correlation < 0.5 or df[similar_column].isnull().any():
            # 使用均值填充
            print(f"列'{column}'均值填充")
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        else:
            # 使用线性回归预测填充
            print(f"列'{column}'线性回归预测填充")
            known_data = df[[column, similar_column]].dropna()
            unknown_data = df[df[column].isnull()]
            
            # 定义特征和目标列
            features = [similar_column]
            target_columns = [column]

            # 划分训练集和测试集
            X_train  = known_data[features] 
            y_train=known_data[target_columns]
            # 训练线性回归模型
            model = LinearRegression()
            model.fit(X_train, y_train)

            # 预测缺失值
            predicted_values = model.predict(unknown_data[features])

            # 填充缺失值
            unknown_data[target_columns] = predicted_values
            filled_data = known_data._append(unknown_data)
            
            df.loc[filled_data.index, [column, similar_column]] = filled_data[[column, similar_column]]

# 遍历每个字段
for key, (first_element, second_element) in similarities.items():
    fill_missing_values(Corr_data, key, first_element, second_element)

# 填充缺失值后检查是否存在负值
negative_values = (Corr_data < 0).any().any()
if negative_values:
    print("存在负值，将使用填充前的数据的平均值来替换。")
    # 填充前的数据
    original_data = hu_data50

    for column in Corr_data.columns:
        # 只对存在负值的列进行处理
        negative_mask = Corr_data[column] < 0
        if negative_mask.any():
            # 使用填充前的数据的平均值替换负值
            mean_value = original_data[column].mean()
            Corr_data.loc[negative_mask, column] = mean_value
# 检查是否还有缺失值
missing_values = Corr_data.isnull().sum().sum()
if missing_values == 0:
    print("数据中不存在任何缺失值。")
    hu_data=Corr_data
else:
    print(f"数据中仍有缺失值。")

列'hid'均值填充
列'chengbaogd'均值填充
列'jygdarea'均值填充
列'jycbgdarea'线性回归预测填充
列'jyczgdarea'均值填充
列'jynhgdarea'线性回归预测填充
列'zcgdarea'均值填充
列'zcqiyearea'均值填充
列'zchezuoshearea'均值填充
列'zccunjitiarea'线性回归预测填充
列'zcnonghuarea'线性回归预测填充
列'zcwainonghuarea'均值填充
列'zcareahetong'线性回归预测填充
列'zcareaqixian'线性回归预测填充
列'jitizcarea'均值填充
列'jitizcareacgb'线性回归预测填充
列'jitizcareazj'均值填充
列'zcareashouzu'均值填充
列'zujin'线性回归预测填充
列'cbdarea'均值填充
列'cbdcwjuli'均值填充
列'cbddljuli'均值填充
列'cychengbaogd'均值填充
存在负值，将使用填充前的数据的平均值来替换。
数据中不存在任何缺失值。


In [11]:
hu_data.to_csv(r"C:\Users\12045\Desktop\户数据(无标准化用于可视化).csv", index=False)

## 5.2.4 标准化处理

In [12]:
from sklearn.preprocessing import StandardScaler

# 提取除了指定列之外的其他列
columns_to_normalize = [col for col in hu_data.columns if col not in["cbdfangwei",
                    "cbdpodu", "zrdpodu", "cbdgaosulu", 
                    "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", 
                    "zrdguangai", "cbdfeili", "zrdfeili", "cbdyongtu201912",
                    "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", 
                    "cbdwrxiufu", "zrdwrxiufu", "shifouzc", "cbdjinqin", 
                    "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao", 
                    "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt",
                    "zrdjypt", "cbdjyptreason1", "zrdjyptreason1",
                    "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", 
                    "zrdjiangqixian", "d301a", "d301b", "d311a", "d311b", 
                    "d312a", "d312b", "d313a", "d313b", "d314a", "d314b", 
                    "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", 
                    "d319a", "d319b", "d320a", "d320b", "d321a", "d321b", 
                    "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", 
                    "d325a", "d325b", "d328a", "d328b", "d329a", "d329b", 
                    "d331a", "d331b", "d335a", "d335b", "d336a", "d336b", 
                    "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", 
                    "d340a", "d340b", "d341a", "d341b","jygdnum","jygdnum1","jygdnum5",
                    "zcgdksnum","zcgdksnum","chengbaogdks"]]

# 对这些列进行标准化处理
scaler = StandardScaler()
hu_data[columns_to_normalize] = scaler.fit_transform(hu_data[columns_to_normalize])

In [13]:
hu_data.to_csv(r"C:\Users\12045\Desktop\户数据(预处理后).csv", index=False)

# 5.3处理地块数据，方法同用户数据

In [14]:
from sklearn.experimental import enable_iterative_imputer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import accuracy_score
import numpy as np

ground_data_without_missing_zujin = ground_data.dropna(subset=["zujin"])
# 用均值填充
mean_imputer = SimpleImputer(strategy='mean')
ground_data_mean = mean_imputer.fit_transform(ground_data_without_missing_zujin)

# 用K近邻插补填充
knn_imputer = KNNImputer()
ground_data_knn = knn_imputer.fit_transform(ground_data_without_missing_zujin)

# 用多变量特征插补填充（多重插补）
iterative_imputer = IterativeImputer(random_state=42)
ground_data_ite = iterative_imputer.fit_transform(ground_data_without_missing_zujin)

# 检验是否全部填充完整
print("Mean Imputation - Missing Values Count:", np.isnan(ground_data_mean).sum())
print("KNN Imputation - Missing Values Count:", np.isnan(ground_data_knn).sum())
print("Iterative Imputation - Missing Values Count:", np.isnan(ground_data_ite).sum())


Mean Imputation - Missing Values Count: 0
KNN Imputation - Missing Values Count: 0
Iterative Imputation - Missing Values Count: 0


In [15]:
# 用学习后的决策树回归器去预测测试集的标签值，和原始数据集的标签“zujin”对比，计算四种方式的MAE
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
# 定义标签列
# 确定 'zujin' 列在第几列
zujin_column_index = None
for i, column in enumerate(ground_data.columns):
    if column == 'zujin':
        zujin_column_index = i
        break

# 提取 'zujin' 列作为标签
labels_mean = ground_data_mean[:, zujin_column_index] 
labels_knn = ground_data_knn[:, zujin_column_index] 
labels_ite = ground_data_ite[:, zujin_column_index] 
# 定义决策树回归器
tree_regressor = DecisionTreeRegressor()

# 使用五折交叉验证计算 Mean Imputation 的 MAE
mae_mean_cv = cross_val_score(tree_regressor, ground_data_mean, labels_mean, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv = -mae_mean_cv.mean()  # 取负号得到正的 MAE

# 使用五折交叉验证计算 KNN Imputation 的 MAE
mae_knn_cv = cross_val_score(tree_regressor,ground_data_knn, labels_knn, cv=10, scoring='neg_mean_absolute_error')
mae_knn_cv = -mae_knn_cv.mean()  # 取负号得到正的 MAE

# 使用五折交叉验证计算 Iterative Imputation 的 MAE
mae_ite_cv = cross_val_score(tree_regressor, ground_data_ite, labels_ite, cv=10, scoring='neg_mean_absolute_error')
mae_ite_cv = -mae_ite_cv.mean()  # 取负号得到正的 MAE

# 输出交叉验证得到的平均绝对误差
print("Mean Imputation MAE :", mae_mean_cv)
print("KNN Imputation MAE :", mae_knn_cv)
print("Iterative Imputation MAE :", mae_ite_cv)


Mean Imputation MAE : 472.8026890756302
KNN Imputation MAE : 449.42626050420165
Iterative Imputation MAE : 470.17546218487394


结果显示地块数据采用均值填充更优

In [16]:
# 计算相关性
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import numpy as np
ground_data = pd.read_csv(r"C:\Users\12045\Desktop\地块数据(预处理中).csv")
# 删除户数据中缺失值占比超过50%的特征列，但保留 "zujin" 列
missing_percentage = (ground_data.isnull().sum() / len(ground_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 50].index
columns_with_high_missing_percentage = columns_with_high_missing_percentage.drop("zujin")
ground_data50 = ground_data.drop(columns=columns_with_high_missing_percentage)
# 删除 "zujin" 列中的缺失值
ground_data50 = ground_data50.dropna(subset=["zujin"])
# 复制数据集
Corr_data = ground_data50.copy()
# 各字段间相关程度
correlation = Corr_data.corr()
# 创建一个空字典来保存每个字段与其相似性最高的另一字段名称及相似性
similarities = {}
# 遍历数据框的列
for column in Corr_data.columns:
    # 计算该列与其他列的相关性
    correlations = Corr_data.corr()[column].drop(column)
    # 找到与当前列相关性最高的字段名和相似性值
    most_similar_column = correlations.idxmax()
    highest_correlation = correlations.max()
    # 保存到字典中
    similarities[column] = (most_similar_column, highest_correlation)
# 输出结果
for column, (most_similar_column, highest_correlation) in similarities.items():
    print(f"字段 '{column}' 与字段 '{most_similar_column}' 的相似性最高，相关性为: {highest_correlation}")

字段 'hid' 与字段 'zrdjyptreason2' 的相似性最高，相关性为: 0.11961624075250007
字段 'year' 与字段 'd329a' 的相似性最高，相关性为: 0.4787478874903556
字段 'chengbaogd' 与字段 'd330b' 的相似性最高，相关性为: 0.7013004589518397
字段 'chengbaogdks' 与字段 'cychengbaogd' 的相似性最高，相关性为: 0.522779895992503
字段 'jygdarea' 与字段 'jynhgdarea' 的相似性最高，相关性为: 0.868035960230705
字段 'jycbgdarea' 与字段 'd330b' 的相似性最高，相关性为: 0.7271250440782292
字段 'jyczgdarea' 与字段 'zcareaqixian' 的相似性最高，相关性为: 0.3299121935967518
字段 'jynhgdarea' 与字段 'jygdarea' 的相似性最高，相关性为: 0.868035960230705
字段 'jygdnum' 与字段 'jygdnum1' 的相似性最高，相关性为: 0.6318008893383532
字段 'jygdnum1' 与字段 'jygdnum' 的相似性最高，相关性为: 0.6318008893383532
字段 'jygdnum5' 与字段 'jynhgdarea' 的相似性最高，相关性为: 0.5272272154981769
字段 'zcgdarea' 与字段 'jitizcarea' 的相似性最高，相关性为: 0.8244847532316224
字段 'zcqiyearea' 与字段 'd323a' 的相似性最高，相关性为: 0.3586414850416256
字段 'zchezuoshearea' 与字段 'zcareahetong' 的相似性最高，相关性为: 0.2502324241919913
字段 'zccunjitiarea' 与字段 'zcgdarea' 的相似性最高，相关性为: 0.5484679015956245
字段 'zcnonghuarea' 与字段 'd315a' 的相似性最高，相关性为: 0.5169247759454143

In [17]:
# 定义填充函数
def fill_missing_values(df, column, similar_column, data):
    if df[column].isnull().any():  # 检查目标列是否有缺失值
        correlation = data

        if correlation < 0.5 or df[similar_column].isnull().any():
            # 使用均值填充
            print(f"列'{column}'均值填充")
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        else:
            # 使用线性回归预测填充
            print(f"列'{column}'线性回归预测填充")
            known_data = df[[column, similar_column]].dropna()
            unknown_data = df[df[column].isnull()]
            
            # 定义特征和目标列
            features = [similar_column]
            target_columns = [column]

            # 划分训练集和测试集
            X_train  = known_data[features] 
            y_train=known_data[target_columns]
            # 训练线性回归模型
            model = LinearRegression()
            model.fit(X_train, y_train)

            # 预测缺失值
            predicted_values = model.predict(unknown_data[features])

            # 填充缺失值
            unknown_data[target_columns] = predicted_values
            filled_data = known_data._append(unknown_data)
            
            df.loc[filled_data.index, [column, similar_column]] = filled_data[[column, similar_column]]

# 遍历每个字段
for key, (first_element, second_element) in similarities.items():
    fill_missing_values(Corr_data, key, first_element, second_element)
# 检查是否还有缺失值
missing_values = Corr_data.isnull().sum().sum()
if missing_values == 0:
    print("数据中不存在任何缺失值。")
else:
    print(f"数据中仍有缺失值。")

列'chengbaogd'均值填充
列'chengbaogdks'线性回归预测填充
列'jygdarea'均值填充
列'jycbgdarea'均值填充
列'jyczgdarea'均值填充
列'jynhgdarea'线性回归预测填充
列'jygdnum'均值填充
列'jygdnum1'线性回归预测填充
列'jygdnum5'线性回归预测填充
列'zcgdarea'均值填充
列'zcqiyearea'均值填充
列'zchezuoshearea'均值填充
列'zccunjitiarea'线性回归预测填充
列'zcnonghuarea'均值填充
列'zcareahetong'线性回归预测填充
列'zcareaqixian'均值填充
列'jitizcarea'均值填充
列'jitizcareacgb'线性回归预测填充
列'cbdfangwei'均值填充
列'zrdfangwei'线性回归预测填充
列'cbdarea'线性回归预测填充
列'cbdcwjuli'均值填充
列'zrdcwjuli'均值填充
列'cbddljuli'均值填充
列'zrddljuli'线性回归预测填充
列'cbdgaosulu'均值填充
列'zrdgaosulu'线性回归预测填充
列'cbdguangai'均值填充
列'zrdguangai'线性回归预测填充
列'cbdfeili'均值填充
列'zrdfeili'线性回归预测填充
列'cbdwrxiufu'均值填充
列'zrdwrxiufu'线性回归预测填充
列'shifouzc'均值填充
列'zrdyear'均值填充
列'zrdsyqixian'均值填充
列'zrdjinqin'均值填充
列'zrdxietiao'均值填充
列'zrdhetong'均值填充
列'zrdjyptreason1'均值填充
列'zrdjyptreason2'线性回归预测填充
列'zrdzuyue'均值填充
列'd302a'均值填充
列'd302b'线性回归预测填充
列'd303a'均值填充
列'd303b'线性回归预测填充
列'd306a'均值填充
列'd306b'线性回归预测填充
列'd307a'均值填充
列'd307b'线性回归预测填充
列'd308a'均值填充
列'd308b'线性回归预测填充
列'd309b'线性回归预测填充
列'd312a'均值填充
列'd312b'

In [18]:
from sklearn.tree import DecisionTreeRegressor
# 提取 'zujin' 列作为标签
labels = ground_data50["zujin"]
# 定义决策树回归器
tree_regressor = DecisionTreeRegressor()

mae_mean_cv_line = cross_val_score(tree_regressor, Corr_data, labels, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv_line = -mae_mean_cv_line.mean()  # 取负号得到正的 MAE

mae_mean_cv50 = cross_val_score(tree_regressor, ground_data50, labels, cv=10, scoring='neg_mean_absolute_error')
mae_mean_cv50 = -mae_mean_cv50.mean()  # 取负号得到正的 MAE
print("均值和线性回归结合填充 :", mae_mean_cv_line)
print("均值填充 :", mae_mean_cv50)

均值和线性回归结合填充 : 315.23808823529413
均值填充 : 2506.432271237877


结果显示采用线性回归和均值填充结合的方法效果更好

最终填充

In [20]:
# 用线性回归结合均值填充
ground_data = pd.read_csv(r"C:\Users\12045\Desktop\地块数据(预处理中).csv")
# 删除户数据中缺失值占比超过50%的特征列，但保留 "zujin" 列
missing_percentage = (ground_data.isnull().sum() / len(ground_data)) * 100
columns_with_high_missing_percentage = missing_percentage[missing_percentage > 50].index
columns_with_high_missing_percentage = columns_with_high_missing_percentage.drop("zujin")
ground_data50 = ground_data.drop(columns=columns_with_high_missing_percentage)
# 对分类数据缺失值采用众数填充
columns_to_fill = ["cbdfangwei", "zrdfangwei", "zjxingshi", "c214b", "cbdpodu", "zrdpodu", "cbdgaosulu",
                   "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", "zrdguangai", "cbdfeili", "zrdfeili",
                   "cbdyongtu201912", "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", "cbdwrxiufu",
                   "zrdwrxiufu", "shifouzc", "cbdjinqin", "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao",
                   "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt", "zrdjypt", "cbdjyptreason1",
                   "zrdjyptreason1", "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", "zrdjiangqixian",
                   "d301a", "d301b", "d311a", "d311b", "d312a", "d312b", "d313a", "d313b", "d314a", "d314b",
                   "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", "d319a", "d319b", "d320a", "d320b",
                   "d321a", "d321b", "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", "d325a", "d325b",
                   "d328a", "d328b", "d329a", "d329b", "d331a", "d331b", "d335a", "d335b", "d336a", "d336b",
                   "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", "d340a", "d340b", "d341a", "d341b"
                   "cbdpodu","zrdpodu","cbdturang","zrdturang","cbdyongtu201912","zrdyongtu201912",
                   "cbdyongtu202008","zrdyongtu202008","cbdjiangqixian","cbdqixian","zrdqixian","d301a","d301b",
                   "d336a","d336b","hid","jygdnum","jygdnum1","jygdnum5",
                    "zcgdksnum","zcgdksnum","chengbaogdks"]

# 填充缺失值
for column in ground_data50:
    if column in columns_to_fill:
        mode_val = ground_data50[column].mode()[0]  # 计算众数并取第一个值
        ground_data50[column].fillna(mode_val, inplace=True)
# 复制数据集
Corr_data = ground_data50.copy()
# 各字段间相关程度
correlation = Corr_data.corr()
# 创建一个空字典来保存每个字段与其相似性最高的另一字段名称及相似性
similarities = {}
# 遍历数据框的列
for column in Corr_data.columns:
    # 计算该列与其他列的相关性
    correlations = Corr_data.corr()[column].drop(column)
    # 找到与当前列相关性最高的字段名和相似性值
    most_similar_column = correlations.idxmax()
    highest_correlation = correlations.max()
    # 保存到字典中
    similarities[column] = (most_similar_column, highest_correlation)
def fill_missing_values(df, column, similar_column, data):
    if df[column].isnull().any():  # 检查目标列是否有缺失值
        correlation = data

        if correlation < 0.5 or df[similar_column].isnull().any():
            # 使用均值填充
            print(f"列'{column}'均值填充")
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        else:
            # 使用线性回归预测填充
            print(f"列'{column}'线性回归预测填充")
            known_data = df[[column, similar_column]].dropna()
            unknown_data = df[df[column].isnull()]
            
            # 定义特征和目标列
            features = [similar_column]
            target_column = column

            # 划分训练集和测试集
            X_train = known_data[[similar_column]]
            y_train = known_data[column]
            
            # 训练线性回归模型
            model = LinearRegression()
            model.fit(X_train, y_train)

            # 预测缺失值
            predicted_values = model.predict(unknown_data[[similar_column]])

            # 填充缺失值
            df.loc[df[column].isnull(), column] = predicted_values

# 遍历每个字段
for key, (first_element, second_element) in similarities.items():
    fill_missing_values(Corr_data, key, first_element, second_element)
    
# 填充缺失值后检查是否存在负值
negative_values = (Corr_data < 0).any().any()
if negative_values:
    print("存在负值，将使用填充前的数据的平均值来替换。")
    # 填充前的数据
    original_data = ground_data50

    for column in Corr_data.columns:
        # 只对存在负值的列进行处理
        negative_mask = Corr_data[column] < 0
        if negative_mask.any():
            # 使用填充前的数据的平均值替换负值
            mean_value = original_data[column].mean()
            Corr_data.loc[negative_mask, column] = mean_value
# 检查是否还有缺失值
missing_values = Corr_data.isnull().sum().sum()
if missing_values == 0:
    print("数据中不存在任何缺失值。")
    ground_data=Corr_data
else:
    print(f"数据中仍有缺失值。")

列'chengbaogd'均值填充
列'jygdarea'均值填充
列'jycbgdarea'均值填充
列'jyczgdarea'均值填充
列'jynhgdarea'线性回归预测填充
列'zcgdarea'均值填充
列'zcqiyearea'均值填充
列'zchezuoshearea'均值填充
列'zccunjitiarea'线性回归预测填充
列'zcnonghuarea'线性回归预测填充
列'zcareahetong'线性回归预测填充
列'zcareaqixian'线性回归预测填充
列'jitizcarea'均值填充
列'jitizcareacgb'线性回归预测填充
列'zujin'均值填充
列'cbdarea'均值填充
列'zrdarea'均值填充
列'cbdcwjuli'均值填充
列'zrdcwjuli'线性回归预测填充
列'cbddljuli'均值填充
列'zrddljuli'均值填充
列'zrdyear'均值填充
列'zrdsyqixian'线性回归预测填充
列'zrdzuyue'均值填充
列'd302a'均值填充
列'd302b'均值填充
列'd303a'均值填充
列'd303b'线性回归预测填充
列'd306a'均值填充
列'd306b'线性回归预测填充
列'd307a'均值填充
列'd307b'线性回归预测填充
列'd308a'均值填充
列'd308b'线性回归预测填充
列'd309b'线性回归预测填充
列'd318a'均值填充
列'd318b'均值填充
列'd327a'均值填充
列'd327b'线性回归预测填充
列'd330a'均值填充
列'd330b'线性回归预测填充
列'd332a'均值填充
列'd332b'线性回归预测填充
列'cychengbaogd'线性回归预测填充
存在负值，将使用填充前的数据的平均值来替换。
数据中不存在任何缺失值。


In [21]:
ground_data.to_csv(r"C:\Users\12045\Desktop\地块数据(无标准化用于可视化).csv", index=False)

In [22]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = [col for col in ground_data.columns if col not in ["cbdfangwei", "zrdfangwei", "zjxingshi", "c214b", "cbdpodu", "zrdpodu", "cbdgaosulu",
                   "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", "zrdguangai", "cbdfeili", "zrdfeili",
                   "cbdyongtu201912", "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", "cbdwrxiufu",
                   "zrdwrxiufu", "shifouzc", "cbdjinqin", "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao",
                   "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt", "zrdjypt", "cbdjyptreason1",
                   "zrdjyptreason1", "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", "zrdjiangqixian",
                   "d301a", "d301b", "d311a", "d311b", "d312a", "d312b", "d313a", "d313b", "d314a", "d314b",
                   "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", "d319a", "d319b", "d320a", "d320b",
                   "d321a", "d321b", "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", "d325a", "d325b",
                   "d328a", "d328b", "d329a", "d329b", "d331a", "d331b", "d335a", "d335b", "d336a", "d336b",
                   "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", "d340a", "d340b", "d341a", "d341b"
                   "cbdpodu","zrdpodu","cbdturang","zrdturang","cbdyongtu201912","zrdyongtu201912",
                   "cbdyongtu202008","zrdyongtu202008","cbdjiangqixian","cbdqixian","zrdqixian","d301a",
                   "d301b","d336a","d336b","hid","jygdnum","jygdnum1","jygdnum5",
                    "zcgdksnum","zcgdksnum","chengbaogdks"]]

# 对这些列进行标准化处理
scaler = StandardScaler()
ground_data[columns_to_normalize] = scaler.fit_transform(ground_data[columns_to_normalize])

In [23]:
ground_data.to_csv(r"C:\Users\12045\Desktop\地块数据(预处理后).csv", index=False)

# 5.4处理村数据，由于村数据缺失值较少，采用任意填充方法区别不大，故使用最简单的均值填充

In [24]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# 使用均值填充缺失值
mean_imputer = SimpleImputer(strategy='mean')
village_data_mean = mean_imputer.fit_transform(village_data)
village_data_mean_df = pd.DataFrame(village_data_mean, columns=village_data.columns.tolist())

# 将数据保存到 CSV 文件中（未标准化）
village_data_mean_df.to_csv(r"C:\Users\12045\Desktop\村数据(无标准化用于可视化).csv", index=False)

# 标准化数据
scaler = StandardScaler()
village_data_scaled = scaler.fit_transform(village_data_mean)

# 将标准化后的数据转换为 DataFrame
village_data_scaled_df = pd.DataFrame(village_data_scaled, columns=village_data.columns.tolist())

# 输出预处理后的数据到 CSV 文件中
village_data_scaled_df.to_csv(r"C:\Users\12045\Desktop\村数据(预处理后).csv", index=False)
